In [1]:
# Get pandas and postgres to work together
import datetime
import pandas.io.sql as pd_sql
import psycopg2 as pg
import pandas as pd

In [2]:
# Postgres info to connect
connection_args = {
    'host': '192.168.0.242',  # We are connecting to our _local_ version of psql
    'dbname': 'tracker-network',    # DB that we are connecting to
    'port': 5432          
}

In [3]:
connection = pg.connect(**connection_args)
connection.autocommit = True

In [4]:
# make a cursor
cursor = connection.cursor()

In [5]:
query = "SELECT * FROM products;"

In [6]:
# pd_sql.read_sql(query, connection)

# make a query (sets the cursor pointing at the first record)
cursor.execute(query)
print(cursor.fetchall())

[(3, '6364253', 'Nintendo - Switch 32GB Console - Gray Joy-Con', 2, 3, 3, 299.99, 'https://api.bestbuy.com/click/-/6364253/pdp', datetime.datetime(2021, 5, 23, 11, 1, 25, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=-420, name=None)), 'https://api.bestbuy.com/click/-/6364253/cart', False), (4, '6401728', 'Nintendo - Switch - Animal Crossing: New Horizons Edition 32GB Console - Multi', 2, 3, 3, 299.99, 'https://api.bestbuy.com/click/-/6401728/pdp', datetime.datetime(2021, 5, 23, 11, 54, 55, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=-420, name=None)), 'https://api.bestbuy.com/click/-/6401728/cart', False), (5, '6460946', 'Nintendo - Switch 32GB Lite - Blue', 2, 3, 3, 199.99, 'https://api.bestbuy.com/click/-/6460946/pdp', datetime.datetime(2021, 5, 23, 11, 54, 55, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=-420, name=None)), 'https://api.bestbuy.com/click/-/6460946/cart', False)]


In [7]:
def product_exists(external_id, vendor_id):
    query = f"SELECT id FROM products WHERE external_id = '{external_id}'"
    cursor.execute(query)
    return cursor.fetchone() is not None

In [8]:
def insert_new_product(external_id, name, vendor_id, category_id, subcategory_id, msrp, product_link, created_at, atc_link, combo):
    query = f"""
    INSERT INTO
        products (external_id, name, vendor_id, category_id, subcategory_id, msrp, product_link, created_at, atc_link, combo)
    VALUES
        ('{external_id}', '{name}', {vendor_id}, {category_id}, {subcategory_id}, {msrp}, '{product_link}', '{created_at}', '{atc_link}', {combo})
    """    
    cursor.execute(query)

In [18]:
def insert_new_drop(external_id, vendor_id, price, created_at):
    query = f"""
    INSERT INTO
        drops (product_id, price, created_at)
    VALUES
        ((SELECT id FROM products WHERE external_id = '{external_id}' and vendor_id = {vendor_id}), {price}, '{created_at}')
    """
    cursor.execute(query)

In [23]:
def process_drops(drops):
    for drop in drops:
        if not product_exists(drop[0], drop[2]):
            insert_new_product(drop[0], drop[1], drop[2], drop[3], drop[4], drop[5], drop[6], drop[7], drop[8], drop[9])

        insert_new_drop(drop[0], drop[2], drop[5], drop[7])

In [24]:
drops = [
    ['6364253', 'Nintendo - Switch 32GB Console - Gray Joy-Con', 2, 3, 3, '299.99', 'https://api.bestbuy.com/click/-/6364253/pdp', '2021/05/23 11:54:55', 'https://api.bestbuy.com/click/-/6364253/cart', False],
    ['6401728', 'Nintendo - Switch - Animal Crossing: New Horizons Edition 32GB Console - Multi', 2, 3, 3, '299.99', 'https://api.bestbuy.com/click/-/6401728/pdp', '2021/05/23 11:54:55', 'https://api.bestbuy.com/click/-/6401728/cart', False],
    ['6460946', 'Nintendo - Switch 32GB Lite - Blue', 2, 3, 3, '199.99', 'https://api.bestbuy.com/click/-/6460946/pdp', '2021/05/23 11:54:55', 'https://api.bestbuy.com/click/-/6460946/cart', False],
    ['51ac336757a4', 'Package - Nintendo - Switch 32GB Lite - Gray and Animal Crossing: New Horizons', 2, 3, 3, '259.98', '/site/combo/nintendo-switch-consoles/e0d0be54-459e-4ea8-900e-51ac336757a4', '2021/05/23 11:54:55', '', True],
    ['3f9814562e56', 'Package - Nintendo - Switch 32GB Lite - Yellow and Animal Crossing: New Horizons', 2, 3, 3, '259.98', '/site/combo/nintendo-switch-consoles/e9b155b6-04c5-4134-bc41-3f9814562e56', '2021/05/23 11:54:55', '', True]
    ]

In [25]:
process_drops(drops)